# Facility Location Problem

Our problem involves transporting supply from facilities to consumers, for example shipping products from factories to retail locations. Let there be $M$ customers and $N$ facilities. Each customer $i = 1, \ldots, M$ needs to be shipped $f_i$ units, and each facility $j$ has the capacity to produce $b_j$ units. The cost of transporting one unit of supply from facility $j$ to customer $i$ is given by $d_{ij}$, the distance between them. Our goal is to transport the goods from the facilities to the customers at minimum cost.

The twist is that none of these facilities have been built yet, and we can only build $K$ facilities in total. Goods cannot be shipped from a facility that is not build. This means we also need to decide which facilities to build in order to deliver the goods at minimal cost.

## Problem Formulation

\begin{align*}
\min~~ & \sum_{i=1}^M \sum_{j=1}^N d_{ij} x_{ij} \\
\text{s.t.} ~~ & x_{ij} \leq b_j y_j, && \forall \, i =1,\dots, M, \ j =1,\dots, N, \\
& \sum_{j=1}^N x_{ij} = f_i, && \forall \, i =1,\dots,M,\\
& \sum_{i=1}^M x_{ij} \leq b_j, && \forall \, j =1,\dots,N,\\
& \sum_{j=1}^N y_j \leq K, \\
& x_{ij} \geq 0, && \forall \, i =1,\dots, M, \ j = 1,\dots, N, \\
& y_j \in \{ 0, 1\}, && \forall \, j =1,\dots, N.
\end{align*}

### Decision variables

Let $x_{ij}$ be the supply from facility $j$ to customer $i$ (we have $x_{ij} \ge 0$).

Let $y_j$ be a binary variable indicating if facility $j$ is built.

### Constraints

We need each customer $i$ to receive $f_i$ units of supply:

$\sum_{j=1}^N x_{ij} = f_i, ~~~\forall \, i =1,\dots,M$

We need each facility $j$ to produce at most $b_j$ units of supply:

$\sum_{i=1}^M x_{ij} \leq b_j, ~~~\forall \, i =1,\dots,M$

We restrict ourselves to $K$ facilities being built:

$\sum_{j=1}^N y_j \leq K$

We cannot send supply from facility $j$ unless it is built:

$ x_{ij} \leq b_j y_j,~~~ \forall \, i =1,\dots, M, \ j =1,\dots, N$

This is an example of a big-M constraint: if $y_j = 1$, $x_{ij}$ can be as large as it wants (there is no way it can be higher than $b_j$ though due to the supply constraint above), and if $y_j = 0$, then $x_{ij}$ is forced to be zero.

### Objective

We want to minimize the total cost:

$\min \sum_{i=1}^M \sum_{j=1}^N d_{ij} x_{ij}$


## Part I: Solving the problem in JuMP

Suppose we live in a 1D world, where customers and facilities are located on a line, and $d_{ij}$ is just the distance between customer $i$ and facility $j$ on the line. Let's solve the problem for the following data:

In [2]:
customer_locs = [3 1; 7 2; 4 3; 0 6; 4 7; 15 2; 5 10; 8 15]
facility_locs = [1 1; 5 4; 6 10; 12 2; 15 15]
f = [1; 1; 6; 8; 6; 10; 6; 10]
b = [22; 18; 15; 21; 23]
K = 3
M = size(customer_locs,1)
N = size(facility_locs,1)

5

Calculate distances:

In [3]:
d = [norm(customer_locs[i,:] - facility_locs[j,:]) for i = 1:M, j = 1:N]

8×5 Array{Float64,2}:
  2.0       3.60555   9.48683   9.05539  18.4391
  6.08276   2.82843   8.06226   5.0      15.2643
  3.60555   1.41421   7.28011   8.06226  16.2788
  5.09902   5.38516   7.2111   12.6491   17.4929
  6.7082    3.16228   3.60555   9.43398  13.6015
 14.0357   10.198    12.0416    3.0      13.0   
  9.84886   6.0       1.0      10.6301   11.1803
 15.6525   11.4018    5.38516  13.6015    7.0   

Let's make a model in JuMP!

In [4]:
using JuMP, Gurobi
model = Model(solver=GurobiSolver(OutputFlag=0))

# Add variables
@variable(model, x[i = 1:M, j = 1:N] >= 0)
@variable(model, y[j = 1:N], Bin)

# Add constraints
@constraint(model, mustbuild[i = 1:M, j = 1:N], x[i, j] <= b[j] * y[j])
@constraint(model, demand[i = 1:M], sum(x[i, j] for j = 1:N) == f[i])
@constraint(model, sum(y[j] for j = 1:N) <= K)

# Add objective
@objective(model, Min, sum(x[i, j] * d[i, j] for i = 1:M, j = 1:N))

2 x[1,1] + 3.605551275463989 x[1,2] + 9.486832980505138 x[1,3] + 9.055385138137417 x[1,4] + 18.439088914585774 x[1,5] + 6.082762530298219 x[2,1] + 2.8284271247461903 x[2,2] + 8.06225774829855 x[2,3] + 5 x[2,4] + 15.264337522473747 x[2,5] + 3.605551275463989 x[3,1] + 1.4142135623730951 x[3,2] + 7.280109889280518 x[3,3] + 8.06225774829855 x[3,4] + 16.278820596099706 x[3,5] + 5.0990195135927845 x[4,1] + 5.385164807134504 x[4,2] + 7.211102550927978 x[4,3] + 12.649110640673518 x[4,4] + 17.4928556845359 x[4,5] + 6.708203932499369 x[5,1] + 3.1622776601683795 x[5,2] + 3.605551275463989 x[5,3] + 9.433981132056603 x[5,4] + 13.601470508735444 x[5,5] + 14.035668847618199 x[6,1] + 10.198039027185569 x[6,2] + 12.041594578792296 x[6,3] + 3 x[6,4] + 13 x[6,5] + 9.848857801796104 x[7,1] + 6 x[7,2] + x[7,3] + 10.63014581273465 x[7,4] + 11.180339887498949 x[7,5] + 15.652475842498529 x[8,1] + 11.40175425099138 x[8,2] + 5.385164807134504 x[8,3] + 13.601470508735444 x[8,4] + 7 x[8,5]

In [5]:
# Solve the model
status = solve(model)
status

:Optimal

Academic license - for non-commercial use only


In [6]:
# Check the optimal variable values
getvalue(x)

8×5 Array{Float64,2}:
 0.0  1.0   0.0   0.0  0.0
 0.0  1.0   0.0   0.0  0.0
 0.0  6.0   0.0   0.0  0.0
 0.0  8.0   0.0   0.0  0.0
 0.0  6.0   0.0   0.0  0.0
 0.0  0.0   0.0  10.0  0.0
 0.0  0.0   6.0   0.0  0.0
 0.0  0.0  10.0   0.0  0.0

In [7]:
getvalue(y)

5-element Array{Float64,1}:
  0.0
  1.0
  1.0
  1.0
 -0.0

In [8]:
# Check the optimal objective value
getobjectivevalue(model)

166.8258922638801